## Prerequisities

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 14.2 MB/s 
     |████████████████████████████████| 3.3 MB 86.3 MB/s 
     |████████████████████████████████| 636 kB 39.4 MB/s 
     |████████████████████████████████| 895 kB 66.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertForMaskedLM
from transformers import BertTokenizer
from transformers import pipeline

In [ ]:
import random

##Source

In [ ]:
def get_pred_model():
    # model = BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
    # tokenizer = BertTokenizer.from_pretrained(
    #     "dkleczek/bert-base-polish-uncased-v1"
    # )
    model = BertForMaskedLM.from_pretrained("allegro/herbert-large-cased")
    tokenizer = BertTokenizer.from_pretrained(
        "allegro/herbert-large-cased"
    )
    pred_pipeline = pipeline("fill-mask", model=model, tokenizer=tokenizer)
    return pred_pipeline

In [ ]:
from transformers import AutoTokenizer, AutoModel

def get_pred_model():
    # model = BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
    # tokenizer = BertTokenizer.from_pretrained(
    #     "dkleczek/bert-base-polish-uncased-v1"
    # )
    model = BertForMaskedLM.from_pretrained("allegro/herbert-large-cased")
    # tokenizer = BertTokenizer.from_pretrained(
    #     "allegro/herbert-large-cased"
    # )
    tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-large-cased")
    pred_pipeline = pipeline("fill-mask", model=model, tokenizer=tokenizer)
    return pred_pipeline

In [ ]:
class WordGenerator:
    """
    Class to extend given sentence
    """

    def __init__(self):
        self.model = get_pred_model()

    def add_punctuation(self, user_input):
        user_input = user_input.strip()
        if user_input[-1] not in "?!.":
            user_input += "."
        return user_input

    def choose_sent(self, words, sequences_words, scores):
        chosen = random.choices(sequences_words, scores)[0]
        new_word = chosen[1]
        if new_word not in words:
            res = " ".join(chosen[0])
        return " ".join(words)

    def generate_in(self, user_input, start_words, add_word_num):
        user_input = self.add_punctuation(user_input)
        result = start_words

        for i in range(add_word_num):
            words = result.split()
            if len(words) == 1:
                mask_index = 1
            else:
                mask_index = random.randint(1, len(words) - 1)
            words.insert(mask_index, self.model.tokenizer.mask_token)
            preds = self.model(f"{user_input} {' '.join(words)}")
            sequences_words = [
                (pred["sequence"].split()[-len(words) - 1 : -1], pred["token_str"])
                for pred in preds
            ]
            scores = [pred["score"] for pred in preds]
            result = self.choose_sent(result.split(), sequences_words, scores)

        return result

In [ ]:
word_generator = WordGenerator()

Downloading:   0%|          | 0.00/664 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Some weights of the model checkpoint at allegro/herbert-large-cased were not used when initializing BertForMaskedLM: ['cls.sso.sso_relationship.bias', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/907k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/556k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129 [00:00<?, ?B/s]

In [ ]:
word_generator.generate_in("Lubię truskawki", "Truskawki owoce", 3)

'truskawki. truskawki. Lubię codziennie jeść'

##Od nowa

In [ ]:
!pip install transformers

In [ ]:
!pip install morfeusz2

     |████████████████████████████████| 8.2 MB 27.9 MB/s 


In [ ]:
import morfeusz2
morf = morfeusz2.Morfeusz()

In [ ]:
from transformers import pipeline
from transformers import BertForMaskedLM
from transformers import BertTokenizer

model = BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
tokenizer = BertTokenizer.from_pretrained(
    "dkleczek/bert-base-polish-uncased-v1")
nlp = pipeline('fill-mask', model=model, tokenizer=tokenizer)

Some weights of the model checkpoint at dkleczek/bert-base-polish-uncased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import string

In [ ]:
for pred in nlp(f"Adam Mickiewicz wielkim polskim {nlp.tokenizer.mask_token} był."):
    print(pred) 

{'sequence': 'adam mickiewicz wielkim polskim poeta był.', 'score': 0.4719655513763428, 'token': 26596, 'token_str': 'p o e t a'}
{'sequence': 'adam mickiewicz wielkim polskim bohaterem był.', 'score': 0.09128003567457199, 'token': 10953, 'token_str': 'b o h a t e r e m'}
{'sequence': 'adam mickiewicz wielkim polskim człowiekiem był.', 'score': 0.06471816450357437, 'token': 5182, 'token_str': 'c z ł o w i e k i e m'}
{'sequence': 'adam mickiewicz wielkim polskim pisarzem był.', 'score': 0.05232352018356323, 'token': 24293, 'token_str': 'p i s a r z e m'}
{'sequence': 'adam mickiewicz wielkim polskim politykiem był.', 'score': 0.045542340725660324, 'token': 44095, 'token_str': 'p o l i t y k i e m'}


In [ ]:
for pred in nlp(f"Lubię {nlp.tokenizer.mask_token} je czytać."):
    print(pred)

{'sequence': 'lubie tez je czytac.', 'score': 0.2278040200471878, 'token': 13654, 'token_str': 't e z'}
{'sequence': 'lubie ci je czytac.', 'score': 0.16950087249279022, 'token': 2052, 'token_str': 'c i'}
{'sequence': 'lubie tylko je czytac.', 'score': 0.06490723788738251, 'token': 2098, 'token_str': 't y l k o'}
{'sequence': 'lubie moc je czytac.', 'score': 0.059427645057439804, 'token': 3735, 'token_str': 'm o c'}
{'sequence': 'lubie sobie je czytac.', 'score': 0.03328728303313255, 'token': 2281, 'token_str': 's o b i e'}


In [ ]:
def generate_in(sentence):
  words = sentence.split()

  # remove punctuation
  words = [w.translate(str.maketrans('', '', string.punctuation)) for w in words]
  
  